In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

In [2]:
# #SET GROUPS FOR THE CORE-SHELLS
# group lithium type 1
# group coreshell type 2 3 4 5 6 7 8 9
# group cores type  2 4 6 8
# group shells type 3 5 7 9

# #SET BOND_STYLE UNDER PAIR STYLE/COEFF
# bond_style harmonic
# bond_coeff 1  93.7     0.0
# bond_coeff 2  196.3    0.0
# bond_coeff 3  95.0     0.0
# bond_coeff 4  65.0     0.0

# #ADDITIONAL COMPUTE COMMANDS????
# compute         cs_temp all temp/cs cores shells             #computes temperature of the coreshells
# compute         cs_press all pressure cs_temp                #computes pressure based on coreshell temp
# velocity        all create ${T1} 100 bias yes temp cs_temp   #velocity not including relative motion but only centre mass of cs pair
# velocity        all scale ${T1} temp cs_temp                 #velocity scaled by temp include cs modification


In [3]:
elements = ['Li', 'Ni', 'OC', 'OS']
li, ni, oc, os = elements

pairs = [(li,os), (ni,os), (os,os)]

charges = {li: 1, ni: 3, oc: 0.960, os: -2.960}
masses = {li: 6.9410, ni: 52.8237, oc: 14.3991, os: 1.5999}

bpp_def = {'Li_OS_A'   : 456.783,
           'Li_OS_rho' : 0.212,
           'Li_OS_C'   : 0.0,
           'Ni_OS_A'   : 1540.922,
           'Ni_OS_rho' : 0.323,
           'Ni_OS_C'   : 0.425,
           'OS_OS_A'   : 22764.7000,
           'OS_OS_rho' : 0.1490,
           'OS_OS_C'   : 20.4}

bpp = {'Li_OS_A'   : 456.783,
       'Li_OS_rho' : 0.212,
       'Li_OS_C'   : 0.0,
       'Ni_OS_A'   : 1540.922,
       'Ni_OS_rho' : 0.323,
       'Ni_OS_C'   : 0.425,
       'OS_OS_A'   : 22764.7000,
       'OS_OS_rho' : 0.1490,
       'OS_OS_C'   : 20.4}


In [4]:
structures = []
for structure in glob.glob('poscars/POSCAR*'):
    st = pmg.Structure.from_file(structure)
    structures.append(st)

In [ ]:
def init(structure):
    lmp = lammps.Lammps(units='metal',
                        style = 'full',
                        args=['-log', 'none', '-screen', 'none'])
    lmp.system.add_pymatgen_structure(structure, elements)
    lmp.command('kspace_style pppm 1e-6')
    lmp.command('pair_style buck/coul/long 12.0')
    lmp.command('pair_coeff * * 0 1 0')

    for c in charges:
        lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))
    
    return lmp

In [ ]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
def set_potentials(instance):
    for pair in pairs:

        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]), 
                                      dex(pair[1]),  
                                      bpp['{}_{}_A'.format(pair[0], pair[1])], 
                                      bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                      bpp['{}_{}_C'.format(pair[0], pair[1])]
                                     )
             )
        
def dex(elin): return elements.index(elin) + 1

In [ ]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([instances[0].system.total,3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.run(0)
            out[:,:,instances.index(instance)] = instance.system.forces
            
    else: out = np.ones([instances[0].system.total,3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
        
    return out

In [ ]:
instances = [init(structure) for structure in structures]

In [ ]:
expected = np.zeros([instances[0].system.total,3, len(instances)])

In [ ]:
with pm.Model() as model:
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  100)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
    Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.05)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1), draws=500)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')

60

In [9]:
new_species = []
new_sites = []
[new_species.append(species) for i, species in enumerate(new_structure.species) if species.symbol == "O" ]
[new_sites.append(new_structure.frac_coords[i]) for i, species in enumerate(new_structure.species) if species.symbol == "O" ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [5]:
new_structure = structures[0]

In [7]:
# structures[0].sites
new_species = []
new_sites = []
for i, species in enumerate(new_structure.species):
    if species.symbol == "O":
        new_species.append(species)
        new_sites.append(new_structure.frac_coords[i])

In [10]:
for i, species in enumerate(new_species):
    new_structure.append(new_species[i], new_sites[i], validate_proximity=False, properties=None)

In [11]:
new_structure

Structure Summary
Lattice
    abc : 5.7012844086 12.1410872426 13.9111145166
 angles : 90.0 90.0 90.0
 volume : 962.9244449762201
      A : 5.7012844086 0.0 0.0
      B : 0.0 12.1410872426 0.0
      C : 0.0 0.0 13.9111145166
PeriodicSite: Li (-1.3927, 12.3535, 13.9007) [-0.2443, 1.0175, 0.9993]
PeriodicSite: Li (4.3153, 2.6396, 13.9028) [0.7569, 0.2174, 0.9994]
PeriodicSite: Li (-2.8244, 4.8289, 0.0208) [-0.4954, 0.3977, 0.0015]
PeriodicSite: Li (1.4644, 7.4729, 13.8789) [0.2568, 0.6155, 0.9977]
PeriodicSite: Li (-0.0015, 9.7229, 0.0152) [-0.0003, 0.8008, 0.0011]
PeriodicSite: Li (2.8896, 0.2599, 13.8903) [0.5068, 0.0214, 0.9985]
PeriodicSite: Li (1.5376, 2.4689, 0.0263) [0.2697, 0.2034, 0.0019]
PeriodicSite: Li (0.0592, 4.8118, 0.0126) [0.0104, 0.3963, 0.0009]
PeriodicSite: Li (-1.3953, 7.4876, 13.8993) [-0.2447, 0.6167, 0.9992]
PeriodicSite: Li (-2.8591, 9.6928, 0.0016) [-0.5015, 0.7983, 0.0001]
PeriodicSite: Li (1.4556, 0.9110, 4.6485) [0.2553, 0.0750, 0.3342]
PeriodicSite: Li (0.01